In [1]:
# lib's
import os
import glob

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.preprocessing import image

In [6]:
# Attention Mechanism - Squeeze and Excitation Block
def squeeze_excite_block(input_tensor, ratio=16):
    filters = input_tensor.shape[-1]
    se_shape = (1, 1, filters)

    se = layers.GlobalAveragePooling2D()(input_tensor)
    se = layers.Reshape(se_shape)(se)
    se = layers.Dense(filters // ratio, activation='relu', use_bias=False)(se)
    se = layers.Dense(filters, activation='sigmoid', use_bias=False)(se)

    x = layers.multiply([input_tensor, se])
    return x

# Residual Block with Attention Mechanism
def residual_block(x, filters, kernel_size=3, stride=1, se_block=False):
    shortcut = x
    
    # First Convolution
    x = layers.Conv2D(filters, kernel_size, padding='same', strides=stride)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # Second Convolution
    x = layers.Conv2D(filters, kernel_size, padding='same', strides=1)(x)
    x = layers.BatchNormalization()(x)
    
    # Optional Attention Mechanism
    if se_block:
        x = squeeze_excite_block(x)
    
    print(x.shape)
    print(shortcut.shape)
    # Adding shortcut (Residual Connection)
    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    
    return x

# Custom CNN Model with Attention Mechanisms and Advanced Layers
def custom_cnn_with_SE_attention(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    
    # Initial Convolution Block
    x = layers.Conv2D(32, (3, 3), padding='same', strides=1)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # First Residual Block with SE Attention
    x = residual_block(x, filters=64, kernel_size=3, stride=1, se_block=True)
    
    # Second Residual Block with SE Attention
    x = residual_block(x, filters=128, kernel_size=3, stride=1, se_block=True)
    x = layers.MaxPooling2D((2, 2))(x)
    
    # Third Residual Block without SE Attention
    x = residual_block(x, filters=256, kernel_size=3, stride=1, se_block=False)
    
    # Global Average Pooling for Feature Extraction
    x = layers.GlobalAveragePooling2D()(x)
    
    # Fully Connected Layer with Dropout
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    # Output Layer (for example, for 3-class classification)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    # Build the Model
    model = models.Model(inputs, outputs)
    
    return model


In [7]:
# Step 1: Define the paths for training and validation data
train_data_dir = os.path.join('..', 'Dataset', 'data', 'train')  # Root folder that contains subfolders '0', '1', '2'
test_data_dir = os.path.join('..', 'Dataset', 'data', 'test')  # Root folder that contains subfolders '0', '1', '2'

# Step 2: ImageDataGenerator for loading and augmenting images
# You can apply real-time augmentation if needed
train_datagen = ImageDataGenerator(
    rescale = 1./255, 
    zoom_range = 0.1
)
test_datagen = ImageDataGenerator(rescale = 1./255 )

# Step 3: Load training data (80%) and validation data (20%)
# The flow_from_directory function will infer the labels based on the folder names
train_generator = train_datagen.flow_from_directory(
    directory = train_data_dir,
    target_size = (128, 128),  # Resize all images to the input shape expected by the model
    batch_size = 32,
    class_mode = 'categorical'  # For multi-class classification
)

validation_generator = test_datagen.flow_from_directory(
    directory = test_data_dir,
    target_size = (128, 128),  # Resize all images to the input shape expected by the model
    batch_size = 32,
    class_mode = 'categorical'  # For multi-class classification
)


Found 1206 images belonging to 3 classes.
Found 594 images belonging to 3 classes.


In [8]:
checkpoint_dir = 'checkpoints_CNN_SE_attention_mechanisms_adam'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

checkpoint_filepath = os.path.join(checkpoint_dir,
                                   "model_epoch_{epoch:02d}_val_acc_{val_accuracy:.2f}_val_loss_{val_loss:.2f}.h5")

checkpoint = ModelCheckpoint(filepath = checkpoint_filepath,
                             monitor = 'val_loss',
                             verbose = 0,
                             save_best_only = True,
                             save_weights_only = False,
                             mode = 'auto')

early = EarlyStopping(monitor = 'val_loss',
                      min_delta = 0,
                      patience = 10,
                      verbose = 0,
                      mode = 'auto')

reduceLR = ReduceLROnPlateau(monitor = "val_loss", 
                             factor = 0.1,
                             patience = 5, 
                             verbose = 0, 
                             mode = "auto",
                             min_delta = 0.0001, 
                             cooldown = 0,
                             min_lr = 0.0)

csv_logger = CSVLogger('training.log')

callbacks_list = [checkpoint, reduceLR, csv_logger] #, early

In [9]:
# Step 4: Create the Custom CNN model with Attention Mechanisms
input_shape = (128, 128, 3)  # Example input shape for images
num_classes = len(glob.glob(os.path.join(train_data_dir, '*'))) # 3
model = custom_cnn_with_SE_attention(input_shape, num_classes)

# Step 5: Compile the model
model.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

# Step 6: Train the model
history = model.fit(
    train_generator,  # Training data
    epochs = 100,  # Number of epochs
    validation_data = validation_generator,  # Validation data
    callbacks = callbacks_list
)

# Optional: Save the trained model
save_model_path = os.path.join(checkpoint_dir, 'custom_cnn_SE_attention_model.h5')
model.save(save_model_path)
print(f"Model saved as {save_model_path}.")

(None, 64, 64, 64)
(None, 64, 64, 32)


ValueError: Inputs have incompatible shapes. Received shapes (64, 64, 64) and (64, 64, 32)